<a href="https://colab.research.google.com/github/lucastanger/vehicle_classification/blob/main/vehicle_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install py_dempster_shafer

In [38]:
import pandas as pd

In [39]:
dfs = []

for i in range(1, 7):
  dfs.append(pd.read_csv('https://raw.githubusercontent.com/lucastanger/vehicle_classification/main/data/DE11_{}.csv'.format(i), sep=";"))


dfs

[    Takt Radar / Distanz (m) Breite (m) Reflektionsstaerke Geschwindigkeit
 0      1                61,2      0,735           0,050715          119,56
 1      2                60,8       0,63         0,03803625          119,92
 2      3                60,5       0,42           0,050715          120,64
 3      4                60,4       0,63         0,06339375          119,92
 4      5                60,1       0,63          0,0760725          119,56
 5      6                59,7      0,525          0,0760725          119,56
 6      7                59,3       0,63         0,08875125          119,56
 7      8                58,9       0,63         0,08875125          119,92
 8      9                58,6      0,735            0,10143          119,92
 9     10                58,3      0,067            0,10143          120,28
 10    11                58,1       0,63            0,10143          119,56
 11    12                57,7       0,67         0,08875125          119,92
 12    13   

In [40]:
from pyds import MassFunction

omega = 'plmf'

Vorliegende Daten:

* **Geschwindigkeit** - Mittlere Geschwindigkeiten von 60-90
km/h sprechen für PKW, LKW oder
Motorrad. Höhere Geschwindigkeiten
über 90 km/h sprechen für PKW oder
Motorrad. 

* **Distanz** - Distanz zum vorausfahrenden Objekt - Sehr schnell schwankende Änderungen der Distanz (bei gleichbleibender
Eigengeschwindigkeit, d.h. Beschleunigungen), weisen auf ein Motorrad
hin, schnelle auf einen PKW oder
Motorrad.

* **Breite (m)** - Breiten zwischen 1,60 und 2,15
weisen auf PKW hin, Breiten zwischen 1,90 und 2,50 auf einen LKW
und Breiten zwischen 0,70 und 1,10
auf ein Motorrad oder ein Fahrrad.

* **Reflektionsstaerke** - LKW und PKW liefern eine große
Reflektionsstärke. Motorräder und
Fahrräder liefern deutlich geringere
Reflektionsstärken.

*Die Sensoren arbeiten im Nahbereich recht gut, werden aber bei größeren Entfernungen immer unzuverlässiger* - Diese Information wird zur Bestimmung der Evidenz verwendet.




In [41]:
def speed(speed: float) -> str: 
  if speed <= 60:
    return 'f'
  if 60 < speed < 90:
    return 'plm'
  if speed >= 90:
    return 'pm'

def reflective_strength(reflective_strength: float) -> str:
  if reflective_strength < 0.1:
    return 'fm'
  if 0.1 >= reflective_strength > 0.6:
    return 'pl'
  elif reflective_strength > 0.6:
    return 'l'

def width(width: float) -> str:
  if 1.6 <= width <= 2.15:
    return 'p'
  if 1.9 <= width <= 2.5:
    return 'l'
  if 0.7 <= width <= 1.1:
    return 'fm'
  else:
    return ''

def distance(distance: float, old_distance: float) -> str:
  # Check if ahead driving object gained 5% of distance within 100ms
  if distance > old_distance*1.05:
    return 'm'
  else:
    return ''


Laut Aufgabenbeschreibung funktionieren die Sensoren im Nahbereich gut und werden auf Entfernung immer schlechter. 

**Annahme:** Die Evidenz steigt in Abhängigkeit der Distanz:

Evidenz = 1 - Distanz/2 /100

Omega = 1 - Evidenz



In [42]:
def compute_evidence(distance: float) -> float:
  return 1-distance/2/100

In [43]:
def createMassFunction(om: list, distance: float) -> MassFunction:
  """ Wrapper function for creation of pyds MassFunction """
  # Remove empty elements
  om = list(filter(None, om)) 
  # Remove duplicates
  om = list(dict.fromkeys(om))
  # Create param list
  param = {}

  for elem in om:
    param[elem] = compute_evidence(distance=distance)/len(om)

  param[omega] = 1 - compute_evidence(distance=distance)

  m = MassFunction(param)

  return m

In [45]:
old_dist = 0

mass_functions = []

for row in dfs[5].iloc:
  omega_ = []
  for i in range(1,5):
    s = row[i]
    s = s.replace(',', '.')
    if i == 1:
      if old_dist == 0:
        omega_.append(distance(float(s), float(s)))
        old_dist = float(s)
      else:
        omega_.append(distance(float(s), old_dist))
        old_dist = float(s)
    elif i == 2:
      omega_.append(width(float(s)))
    elif i == 3:
      omega_.append(reflective_strength(float(s)))
    elif i == 4:
      omega_.append(speed(float(s)))

  print(f'Takt {row[0]}: {omega_}')
  mass_functions.append(createMassFunction(omega_, float(row[1].replace(',', '.'))))

Takt 1: ['', 'p', 'pl', 'plm']
Takt 2: ['', 'p', 'pl', 'plm']
Takt 3: ['', 'l', 'pl', 'plm']
Takt 4: ['', 'p', 'pl', 'plm']
Takt 5: ['', 'p', 'pl', 'plm']
Takt 6: ['', 'p', 'pl', 'plm']
Takt 7: ['', 'p', 'pl', 'plm']
Takt 8: ['', 'p', 'pl', 'plm']
Takt 9: ['', 'p', 'pl', 'plm']
Takt 10: ['', 'p', 'pl', 'plm']
Takt 10: ['', 'p', 'pl', 'plm']
Takt 10: ['', 'p', 'pl', 'plm']


In [46]:
c = None

for i, func in enumerate(mass_functions): 
  if i == 0:
    c = func
  else:
    c = c.combine_conjunctive(mass_function=mass_functions[i-1])
  print(f'{c}')

print(f'Final Massfunction: {c}')

{{'m', 'p', 'f', 'l'}:0.375; {'m', 'p', 'l'}:0.20833333333333334; {'p', 'l'}:0.20833333333333334; {'p'}:0.20833333333333334}
{{'p'}:0.3732638888888889; {'p', 'l'}:0.28645833333333337; {'m', 'p', 'l'}:0.1996527777777778; {'m', 'p', 'f', 'l'}:0.140625}
{{'p'}:0.5043561921296296; {'p', 'l'}:0.29771556712962965; {'m', 'p', 'l'}:0.14554542824074074; {'m', 'p', 'f', 'l'}:0.05238281250000001}
{{'p'}:0.4456414130746644; {'p', 'l'}:0.30954569955463324; {'l'}:0.11641530325432045; {'m', 'p', 'l'}:0.10671996252894599; {'m', 'p', 'f', 'l'}:0.02167762158743591}
{{'p'}:0.5516824638847276; {'p', 'l'}:0.2781305478744466; {'l'}:0.09414668258420715; {'m', 'p', 'l'}:0.0678952266097409; {'m', 'p', 'f', 'l'}:0.008145079046877677}
{{'p'}:0.6396016199579234; {'p', 'l'}:0.24004865578336765; {'l'}:0.07568365980883608; {'m', 'p', 'l'}:0.041644982750394036; {'m', 'p', 'f', 'l'}:0.003021081699478893}
{{'p'}:0.7118339221743443; {'p', 'l'}:0.2016224008201719; {'l'}:0.06051487634338769; {'m', 'p', 'l'}:0.024923363022

In [47]:
def compute_plausibility(mass_function: MassFunction) -> None:
  for elem in omega:
    print(f'Plausibility of {elem}: {mass_function.pl({elem})}')

def compute_belief(mass_function: MassFunction) -> None:
  for elem in omega:
    print(f'B({elem}): {mass_function.bel({elem})}')

  print(f'B(four wheels) = B(p,l) = {c.bel("pl")}')
  print(f'B(two wheels) = B(f,m) = {c.bel("fm")}')

def compute_doubt(mass_function: MassFunction, lbl: str) -> None: 
  print(f'Doubt on classification of {lbl}: {mass_function.bel(omega.replace(lbl, ""))}')

In [48]:
compute_plausibility(c)

Plausibility of p: 0.9814886190686498
Plausibility of l: 0.08814943300642952
Plausibility of m: 0.0015571307253271676
Plausibility of f: 5.771897353117229e-06


In [49]:
compute_belief(c)

B(p): 0.9118505669935705
B(l): 0.0185113809313502
B(m): 0.0
B(f): 0.0
B(four wheels) = B(p,l) = 0.9984428692746729
B(two wheels) = B(f,m) = 0.0


In [50]:
compute_doubt(c, 'p')

Doubt on classification of p: 0.0185113809313502


In [51]:
for i, fun in enumerate(mass_functions):
  print(f'MassFuntion at {i*100}ms: {fun}')

MassFuntion at 0ms: {{'m', 'p', 'f', 'l'}:0.375; {'m', 'p', 'l'}:0.20833333333333334; {'p', 'l'}:0.20833333333333334; {'p'}:0.20833333333333334}
MassFuntion at 100ms: {{'m', 'p', 'f', 'l'}:0.37250000000000005; {'m', 'p', 'l'}:0.20916666666666664; {'p', 'l'}:0.20916666666666664; {'p'}:0.20916666666666664}
MassFuntion at 200ms: {{'m', 'p', 'f', 'l'}:0.37; {'m', 'p', 'l'}:0.21; {'p', 'l'}:0.21; {'l'}:0.21}
MassFuntion at 300ms: {{'m', 'p', 'f', 'l'}:0.36650000000000005; {'m', 'p', 'l'}:0.21116666666666664; {'p', 'l'}:0.21116666666666664; {'p'}:0.21116666666666664}
MassFuntion at 400ms: {{'m', 'p', 'f', 'l'}:0.36349999999999993; {'m', 'p', 'l'}:0.2121666666666667; {'p', 'l'}:0.2121666666666667; {'p'}:0.2121666666666667}
MassFuntion at 500ms: {{'m', 'p', 'f', 'l'}:0.36; {'m', 'p', 'l'}:0.21333333333333335; {'p', 'l'}:0.21333333333333335; {'p'}:0.21333333333333335}
MassFuntion at 600ms: {{'m', 'p', 'f', 'l'}:0.357; {'m', 'p', 'l'}:0.21433333333333335; {'p', 'l'}:0.21433333333333335; {'p'}:0.